In [1]:
import pandas as pd
import re
import plotly.express as px
import numpy as np
import plotly.graph_objects as go


In [2]:
excel_data = pd.ExcelFile("OECD-Health-Statistics-2022-Frequently-Requested-Data.xls")

In [3]:
# Data exploration

data = dict()
data_long = pd.DataFrame()
for n in excel_data.sheet_names:
    print(n)
    data[n] = pd.read_excel(excel_data, sheet_name=n, skiprows=3).replace({'..': None})
    first_col_name = data[n].columns[0]
    long_temp=pd.melt(data[n], id_vars=first_col_name).rename(columns={first_col_name: 'country', 'variable': 'year'}, errors='ignore').dropna()
    long_temp['variable']=n
    long_temp=long_temp[['country', 'year', 'variable', 'value']]
    if len(data_long)==0:
        data_long = long_temp
    else:
        data_long = pd.concat([data_long, long_temp])

Frequently Requested Data
Current expenditure, % GDP
Curr exp, per capita US$PPP
Current exp, per capita, growth
GovtCompIns, % curr exp
GovtCompIns, per capita US$PPP
GovtCompIns, per capita, growth
OOP payments, % curr exp
OOP payments, per capita US$PPP
Pharmaceutical exp, % curr exp
Pharma exp, per capita US$PPP
Physicians
Nurses
Medical graduates
Nursing graduates
Hospital beds
Psychiatric care beds
MRI
CT scanners 
Doctor consultations
MRI exams
CT exams
Inpatient care discharges
ALOS, inpatient care
ALOS, AMI
ALOS, delivery
C-sections
Antibiotics
LE Females at birth
LE Males at birth
LE Total population at birth
LE Females at 65
LE Males at 65
Infant mortality rate
PYLL, All causes, Females
PYLL, All causes, Males 
Suicides, deaths per 100000 pop
Tobacco consumption, females
Tobacco consumption, males
Tobacco consumption, total
Alcohol consumption
Obesity, females (SR)
Obesity, males (SR)
Obesity, total (SR)
Obesity, females (M)
Obesity, males (M)
Obesity, total (M)


In [4]:
# Adjusting spelling
data_long['country']=data_long['country'].str.replace(r"[^A-Za-z\s]", "").str.replace(r"\s+$", "").replace({"Trkiye": "Türkiye"})

In [5]:
# Year exploration
data_long.loc[~(data_long['year'].astype(str).apply(lambda x: bool(re.match(r"^\d{4}$", x)))), 'year'].value_counts()

year
2020 (or nearest year)    1465
2015-2016                   76
2019-2020                   76
2008-2009                   76
2017-2018                   76
2016-2017                   74
2014-2015                   74
2009-2010                   74
2012-2013                   72
2013-2014                   72
2018-2019                   72
2007-2008                   72
2005-2006                   72
2011-2012                   70
2010-2011                   68
2006 -2007                  38
2020-2021                   38
2006-2007                   38
Name: count, dtype: int64

In [6]:
# Modify year column
data_long = data_long.loc[data_long['year'].astype(str).str.len() < 10]
data_long['year']=data_long['year'].astype(str).str.slice(-4).astype(int)

In [7]:
data_long=data_long.reset_index(drop=True)

In [8]:
# Adjust data type
data_long['value']=data_long['value'].astype(float)

In [9]:
# Filtering data funciton
def prepare_2d_data(df, years, columns):
    output=df.loc[(df['year'].isin(years)) & df['variable'].isin(columns)]
    output=output.pivot(index=['country', 'year'], columns='variable')
    output=output.droplevel(0, axis=1)
    output=output.reset_index()
    return output

In [10]:
# Filter for values of interest
scatter_cols=['Current expenditure, % GDP', 'Physicians', 
              'ALOS, inpatient care']
df=prepare_2d_data(data_long, [2019], scatter_cols)
df['country'] = df['country'].str.replace(r'\s?[\d\¹\²\³]+$', '', regex=True)
aggregated_df = df.groupby(['country', 'year']).agg(lambda x: np.nan if x.isnull().all() else x.dropna().unique()[0])
aggregated_df.reset_index(inplace=True)
aggregated_df


variable,country,year,"ALOS, inpatient care","Current expenditure, % GDP",Physicians
0,Australia,2019,5.3,10.230,3.83
1,Austria,2019,8.3,10.475,5.32
2,Belgium,2019,7.0,10.664,3.16
3,Canada,2019,7.7,10.950,2.74
4,Chile,2019,6.0,9.333,2.64
5,Colombia,2019,NaN,8.117,2.30
6,Costa Rica,2019,6.6,7.221,NaN
7,Czech Republic,2019,9.5,7.605,4.07
8,Denmark,2019,5.7,10.121,4.25
9,Estonia,2019,7.7,6.823,3.47


In [11]:
# Drop missing values
aggregated_df = aggregated_df.dropna()

In [12]:
continents={'Austria' : 'Europe', 'Belgium': 'Europe', 'Canada': 'N. America', 'Finland': 'Europe', 
            'France': 'Europe', 'Germany': 'Europe', 'Iceland': 'Europe', 'Ireland': 'Europe', 'Japan': 'Asia', 
            'Korea': 'Asia', 'New Zealand': 'Oceania', 'Norway': 'Europe', 'Portugal': 'Europe', 'Spain': 'Europe', 
            'Sweden': 'Europe', 'Switzerland': 'Europe', 'United Kingdom': 'Europe', 'United States': 'N. America',
            'Australia': 'Oceania', 'Denmark': 'Europe', 'Netherlands': 'Europe', 'Israel': 'Asia', 'Türkiye': 'Asia',
            'Luxembourg': 'Europe', 'Greece': 'Europe', 'Italy': 'Europe', 'Czech Republic': 'Europe', 
            'Poland': 'Europe', 'Hungary': 'Europe', 'Slovak Republic': 'Europe', 'Estonia': 'Europe', 
            'Mexico': 'N. America', 'Chile': 'S. America', 'Colombia': 'S. America', 'Costa Rica': 'N. America', 
            'Latvia': 'Europe', 'Lithuania': 'Europe', 'Slovenia': 'Europe', 'OECD AVERAGE': 'OECD'}

In [13]:
# Add continents
aggregated_df['continent'] = aggregated_df['country'].map(continents)
aggregated_df.head()

C:\Users\catal\AppData\Local\Temp\ipykernel_14972\2973917855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_df['continent'] = aggregated_df['country'].map(continents)


variable,country,year,"ALOS, inpatient care","Current expenditure, % GDP",Physicians,continent
0,Australia,2019,5.3,10.230,3.83,Oceania
1,Austria,2019,8.3,10.475,5.32,Europe
2,Belgium,2019,7.0,10.664,3.16,Europe
3,Canada,2019,7.7,10.950,2.74,N. America
4,Chile,2019,6.0,9.333,2.64,S. America


In [14]:
# Create a folder for final images
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [15]:
# Create sacatter plot
fig=px.scatter(aggregated_df, x='Current expenditure, % GDP',y='Physicians' , size='ALOS, inpatient care', text="country", color='continent',
               title='Figure 3. Healthcare Expenditure vs. Number of Physicians by Country, Sized by Average Inpatient Stay in 2019')

# Adjust figure size
fig.update_layout(width=1500, height=600) 

# Update axis label
fig.update_layout(yaxis_title='Physicians, density per 1000 population')

fig.show()
fig.write_image("images/fig1.jpeg")

In [16]:
# Create data of 10 years in US

df=prepare_2d_data(data_long, [2099,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019], scatter_cols)
df['country'] = df['country'].str.replace(r'\s?[\d\¹\²\³]+$', '', regex=True)
aggregated_df1 = df.groupby(['country', 'year']).agg(lambda x: np.nan if x.isnull().all() else x.dropna().unique()[0])
aggregated_df1.reset_index(inplace=True)

mask = aggregated_df1['country'] == 'United States'
ag_2 = aggregated_df1[mask]
ag_2


variable,country,year,"ALOS, inpatient care","Current expenditure, % GDP",Physicians
370,United States,2010,6.2,16.196,2.43
371,United States,2011,6.1,16.138,2.46
372,United States,2012,6.1,16.117,2.50
373,United States,2013,6.1,15.987,2.56
374,United States,2014,6.1,16.192,2.58
375,United States,2015,6.1,16.481,2.58
376,United States,2016,6.1,16.793,2.59
377,United States,2017,6.1,16.768,2.61
378,United States,2018,6.1,16.641,2.61
379,United States,2019,6.2,16.676,2.64


In [17]:
# Graph line chart
fig = px.line(ag_2, x='year', y ='Physicians')

fig.show()

In [18]:
# Craeye data set for all countries in the past 10 years

df2=prepare_2d_data(data_long, [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019], scatter_cols)
df2['country'] = df2['country'].str.replace(r'\s?[\d\¹\²\³]+$', '', regex=True)
aggregated_df2 = df2.groupby(['country', 'year']).agg(lambda x: np.nan if x.isnull().all() else x.dropna().unique()[0])
aggregated_df2.reset_index(inplace=True)

aggregated_df2['continent'] = aggregated_df2['country'].map(continents)
aggregated_df2



variable,country,year,"ALOS, inpatient care","Current expenditure, % GDP",Physicians,continent
0,Australia,2009,5.9,8.552,3.12,Oceania
1,Australia,2010,5.8,8.423,NaN,Oceania
2,Australia,2011,5.8,8.540,3.32,Oceania
3,Australia,2012,5.4,8.675,3.31,Oceania
4,Australia,2013,5.3,8.751,3.37,Oceania
...,...,...,...,...,...,...
413,United States,2015,6.1,16.481,2.58,N. America
414,United States,2016,6.1,16.793,2.59,N. America
415,United States,2017,6.1,16.768,2.61,N. America
416,United States,2018,6.1,16.641,2.61,N. America


In [19]:
# Add continents 

mask = aggregated_df2['continent'] == 'N. America'
ag_2 = aggregated_df2[mask]
ag_2.columns

Index(['country', 'year', 'ALOS, inpatient care', 'Current expenditure, % GDP',
       'Physicians', 'continent'],
      dtype='object', name='variable')

In [20]:
# Create separate dataframes for variables of interest

alos = ag_2[['country','year','ALOS, inpatient care']]
gdp = ag_2[['country','year','Current expenditure, % GDP']]
phys = ag_2[['country','year','Physicians']]
phys = phys.dropna()
alos.head(1)

variable,country,year,"ALOS, inpatient care"
33,Canada,2009,7.7


In [21]:
# Create data set of averages

average_alos = alos.groupby('year')['ALOS, inpatient care'].mean().reset_index()
average_gpd = gdp.groupby('year')['Current expenditure, % GDP'].mean().reset_index()
average_phys = phys.groupby('year')['Physicians'].mean().reset_index()

filter_country = ag_2['country'] ==  'United States'
usa = ag_2[filter_country]
average_gpd

,year,"Current expenditure, % GDP"
0,2009,10.22700
1,2010,10.16600
2,2011,10.04425
3,2012,10.01975
4,2013,9.97250
5,2014,9.91625
6,2015,10.13500
7,2016,10.17350
8,2017,10.03325
9,2018,10.03900


In [22]:
# Create traces
trace1 = go.Scatter(x=usa['year'], y=usa['Current expenditure, % GDP'], mode='lines+markers', name='USA ')
trace2 = go.Scatter(x=average_gpd['year'], y=average_gpd['Current expenditure, % GDP'], mode='lines+markers', name='Continent Average', marker=dict(color='black'))

# Create the figure
fig = go.Figure(data=[trace1, trace2])

# Update layout
fig.update_layout(title='Current Expenditure on Health, % GDP Comparison: United States vs. Continent Average',
                  xaxis_title='Year',
                  yaxis_title='% GDP',
                  legend_title='Data')

# Show the figure
fig.show()


In [23]:
# Create traces
trace1 = go.Scatter(x=usa['year'], y=usa['ALOS, inpatient care'], mode='lines+markers', name='USA')
trace2 = go.Scatter(x=average_alos['year'], y=average_alos['ALOS, inpatient care'], mode='lines+markers', name='Continent Average', marker=dict(color='black'))

# Create the figure
fig = go.Figure(data=[trace1, trace2])

# Update layout
fig.update_layout(title='Average Length of Stay (ALOS) Comparison: United States vs. Continent Average',
                  xaxis_title='Year',
                  yaxis_title='ALOS, inpatient care days',
                  legend_title='Data')

# Show the figure
fig.show()



In [24]:
# Create traces
trace1 = go.Scatter(x=usa['year'], y=usa['ALOS, inpatient care'], mode='lines+markers', name='USA')
trace2 = go.Scatter(x=average_phys['year'], y=average_phys['Physicians'], mode='lines+markers', name='Continent Average',  marker=dict(color='black'))

# Create the figure
fig = go.Figure(data=[trace1, trace2])

# Update layout
fig.update_layout(title='Physicians Comparison: United States vs. Continent Average from 2009 to 2019',
                  xaxis_title='Year',
                  yaxis_title='Physicians, density per 1000 population',
                  legend_title='Data')

# Show the figure
fig.show()
fig.write_image("images/fig2.jpeg")

In [25]:
# Create dash application

import dash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc




fig7 = px.scatter(aggregated_df, x='Current expenditure, % GDP', y='Physicians', size='ALOS, inpatient care', text="country", color='continent')

app = Dash(__name__, external_stylesheets=[dbc.themes.LITERA])

# Define app layout with an additional graph on top
app.layout = html.Div([
    html.H3('Scatter Plot of Healthcare Metrics'),
    dcc.Graph(
        id='scatter-plot',
        figure=fig7
    ),
    html.H3('Metric Comparison Over Time'),
    dcc.Dropdown(
        id='yaxis-column',
        options=[
            {'label': 'ALOS, inpatient care', 'value': 'ALOS, inpatient care'},
            {'label': 'Current expenditure, % GDP', 'value': 'Current expenditure, % GDP'},
            {'label': 'Physicians', 'value': 'Physicians'}
        ],
        value='ALOS, inpatient care'  # Default value
    ),
    dcc.Graph(id='indicator-graphic')
])


# Define callback to update graph
@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('yaxis-column', 'value')]
)
def update_graph(yaxis_column_name):
    if yaxis_column_name == 'ALOS, inpatient care':
        average_df = average_alos
    elif yaxis_column_name == 'Current expenditure, % GDP':
        average_df = average_gpd
    else:
        average_df = average_phys

    trace1 = go.Scatter(x=usa['year'], y=usa[yaxis_column_name], mode='lines+markers', name='USA')
    trace2 = go.Scatter(x=average_df['year'], y=average_df[yaxis_column_name], mode='lines+markers', name='Continent Average')

    fig = go.Figure(data=[trace1, trace2])
    fig.update_layout(title=f'{yaxis_column_name} Comparison: United States vs. Continent Average',
                      xaxis_title='Year',
                      yaxis_title=yaxis_column_name,
                      legend_title='Data')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)